In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output
from time import sleep
import os
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip -u ../input/facial-keypoints-detection/test.zip
!unzip -u ../input/facial-keypoints-detection/training.zip

In [ ]:
train_file = 'training.csv'
test_file = 'test.csv'
lookup_file = '../input/facial-keypoints-detection/IdLookupTable.csv'
train = pd.read_csv(train_file)
test = pd.read_csv(test_file)
lookup = pd.read_csv(lookup_file)

In [ ]:
train.head()

In [ ]:
lookup.head().T

#### Missin Values

In [ ]:
train.isnull().any().value_counts()

Handling Missing Values

In [ ]:
train.fillna(method = 'ffill',inplace = True)
train.isnull().any().value_counts()
train.shape

#### Let's separate the labels and features. 

In [ ]:
imag = []
for i in range(0,7049):
    img = train['Image'][i].split(' ')
    img = ['0' if x == '' else x for x in img]
    imag.append(img)

In [ ]:
imag[0]

**Let's reshape and convert it into float value.**

In [ ]:
image_list = np.array(imag,dtype = 'float')
X_train = image_list.reshape(-1,96,96,1)

**Sample Image**

In [ ]:
plt.imshow(train[10].reshape(96,96),cmap='gray')
plt.show()

**Let's seperate labels**

In [ ]:
training = train.drop('Image',axis = 1)

y_train = []
for i in range(0,7049):
    y = training.iloc[i,:]

    y_train.append(y)
y_train = np.array(y_train,dtype = 'float')

## Implementation with Keras

  1. Convolution Operation
* The convulution operation is one of the fundemantal building a CNN.
* We have a input matrix(the input picture) and a filter(feature detector).
* Filter usally is a 3x3 matrix but it is not a rule.
* Filter detects horizantal or vertical lines and convex shape on the picture. For example in a person picture, we can find ears or noise etc.



**Padding:**

 After edge detection we need to use padding. In edge detection step, we saw that If we use 6x6 input and 3x3 filter then we end up with a 4x4 matrix. Everytime we apply convolution operation then out image shrinks. If a covolution operation as above is applied, we can repeat this operation two or three time because our image getting starts really small. So we ara throwing awat information near the edge of image. To solve this proble, we can pad the image. If we pad additional one border 6x6 image, we get 8x8 image instead of 6x6 image. After padding, we appyle 3x3 filter again we end up with 6x6 matrix. So, we preserve the original input size.



2. Pooling Operation

 We apply pooling to reduce the size of network and speed the computation. We can apply avarage pooling or max pooling. Let's suppose we have 4x4 input matrix, If we apply max pooling then the output will be 2x2 matrix. The way you do that is really simple. It has two hyperparameters, filter size(f) and stride(s).
 

3. Flattening

 Flattening is converting the output of convolutional layers into a 1 dimensional array for inputing it to next layer. It is connected to fully connected layer.

In [ ]:
from keras.layers.advanced_activations import ReLU
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D

In [ ]:
model = Sequential()

model.add(Convolution2D(32, (3,3), activation = 'relu', padding='same', use_bias=False, input_shape=(96,96,1)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3,3), activation = 'relu', padding='same', use_bias=False))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3,3), activation = 'relu', padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(128, (3,3), activation = 'relu', padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30))
model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['acc'])

In [ ]:
model.fit(X_train,y_train,epochs = 10,batch_size = 32,validation_split = 0.2)

#### Preparing Test Data

In [ ]:
timag = []
for i in range(0,1783):
    timg = test['Image'][i].split(' ')
    timg = ['0' if x == '' else x for x in timg]    
    timag.append(timg)
    
    
# reshape and convert

timage_list = np.array(timag,dtype = 'float')
X_test = timage_list.reshape(-1,96,96,1) 



#### Prediction

In [ ]:
pred = model.predict(X_test)

**Creating submission**



In [ ]:
lookid_list = list(lookup['FeatureName'])
imageID = list(lookup['ImageId']-1)
pre_list = list(pred)

rowid = lookup['RowId']
rowid=list(rowid)

feature = []
for f in list(lookup['FeatureName']):
    feature.append(lookid_list.index(f))
    

preded = []
for x,y in zip(imageID,feature):
    preded.append(pre_list[x][y])
rowid = pd.Series(rowid,name = 'RowId')
loc = pd.Series(preded,name = 'Location')
submission = pd.concat([rowid,loc],axis = 1)
submission.to_csv('face_key_submission.csv',index = False)